In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [3]:
labels = df[1].values

In [4]:
type(labels)

numpy.ndarray

In [5]:
texts = df[0].values.tolist()

In [6]:
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification

In [7]:
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
# xlarge version => "albert-xlarge-v2"
# xxlarge version => "albert-xxlarge-v2"

In [8]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(labels)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, y_one_hot, test_size=0.2, random_state=0)

In [10]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

In [11]:
model = TFAlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
# xlarge version => "albert-xlarge-v2"
# xxlarge version => "albert-xxlarge-v2"

All model checkpoint layers were used when initializing TFAlbertForSequenceClassification.

Some layers of TFAlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
optimizer = tf.keras.optimizers.Adam(2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
checkpoint_filepath = "./checkpoints/checkpoint_albert_en"
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_best_only=True, save_weights_only=True)

In [14]:
model.fit(dict(X_train_tokenized), y_train, epochs=10, batch_size=128, validation_split=0.1, callbacks=[es, mc])

Epoch 1/10
39/39 [==============================] - 227s 6s/step - loss: 0.5657 - accuracy: 0.7344 - val_loss: 0.5050 - val_accuracy: 0.7744
Epoch 2/10
39/39 [==============================] - 219s 6s/step - loss: 0.3760 - accuracy: 0.8591 - val_loss: 0.3397 - val_accuracy: 0.8646
Epoch 3/10
39/39 [==============================] - 212s 5s/step - loss: 0.2531 - accuracy: 0.9101 - val_loss: 0.3720 - val_accuracy: 0.8484
Epoch 4/10
39/39 [==============================] - 213s 5s/step - loss: 0.1704 - accuracy: 0.9464 - val_loss: 0.3226 - val_accuracy: 0.8845
Epoch 5/10
39/39 [==============================] - 212s 5s/step - loss: 0.1105 - accuracy: 0.9687 - val_loss: 0.3480 - val_accuracy: 0.8809
Epoch 6/10
39/39 [==============================] - 213s 5s/step - loss: 0.0670 - accuracy: 0.9831 - val_loss: 0.4191 - val_accuracy: 0.8845
Epoch 7/10
39/39 [==============================] - 212s 5s/step - loss: 0.0467 - accuracy: 0.9896 - val_loss: 0.4570 - val_accuracy: 0.8682
Epoch 7: earl

In [15]:
model.load_weights(checkpoint_filepath)
model.evaluate(dict(X_test_tokenized), np.array(y_test))

44/44 [==============================] - 20s 454ms/step - loss: 0.2791 - accuracy: 0.9010


[0.2790921926498413, 0.90101158618927]

In [16]:
y_preds = model.predict(dict(X_test_tokenized))
prediction_probs = tf.nn.softmax(y_preds.logits,axis=1).numpy()
y_predictions = np.argmax(prediction_probs, axis=1)
y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

44/44 [==============================] - 22s 454ms/step
              precision    recall  f1-score   support

           0       0.90      0.89      0.89       657
           1       0.90      0.91      0.91       727

    accuracy                           0.90      1384
   macro avg       0.90      0.90      0.90      1384
weighted avg       0.90      0.90      0.90      1384

